# SYDE 556/750 --- Assignment 1

**Student ID: 20709541**

_Note:_ Please include your numerical student ID only, do _not_ include your name.

_Note:_ Refer to the [PDF](https://github.com/celiasmith/syde556-f22/raw/master/assignments/assignment_01/syde556_assignment_01.pdf) for the full instructions (including some hints), this notebook contains abbreviated instructions only. Cells you need to fill out are marked with a "writing hand" symbol. Of course, you can add new cells in between the instructions, but please leave the instructions intact to facilitate marking.


In [120]:
# Import numpy and matplotlib -- you shouldn't need any other libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize # For question 2.1b)

# Fix the numpy random seed for reproducible results
np.random.seed(18945)

# Some formating options
%config InlineBackend.figure_formats = ['svg']

# 1. Representation of Scalars

## 1.1 Basic encoding and decoding

**a) Computing gain and bias.** In general, for a neuron model $a = G[J]$ (and assuming that the inverse $J = G^{-1}[a]$ exists), solve the following system of equations to compute the gain $\alpha$, and the bias $J^\mathrm{bias}$ given a maximum rate $a^\mathrm{max}$ and an $x$-intercept $\xi$.

$$a^\mathrm{max} = G[\alpha + J^\mathrm{bias}] \,, \quad\quad 0 = G[\alpha \xi + J^\mathrm{bias}] \,.$$


Solving the following system of equations we see that:

$$
\alpha^\mathrm{max} = G[\alpha + J^\mathrm{bias}] \Rightarrow G^{-1}[\alpha^\mathrm{max}]=a+J^\mathrm{bias} \text{ (1)}
$$

and

$$
0 = G[\alpha \xi + J^\mathrm{bias}] \Rightarrow G^{-1}[0]=\alpha \xi +J^\mathrm{bias} \text{ (2)}
$$

from (2) it follows that

$$
J^\mathrm{bias}=G^{-1}[0]-\alpha \xi \text{ (3)}
$$

then plugging (3) into (1) we see that

$$
G^{-1}[a^\mathrm{max}]=\alpha+G^{-1}[0]-\alpha \xi \text{ (4)}
$$

Re-arranging (4) we can find an equation for $\alpha$ as follows

$$
\alpha = \frac{G^{-1}[a^\mathrm{max}]-G^{-1}[0]}{1-\xi} \text{  (5)}
$$

and plugging (5) back into (3) we can find the following equation for $J^\mathrm{bias}$

$$
J^\mathrm{bias}=G^{-1}[0]-\xi \left\{\frac{G^{-1}[a^\mathrm{max}]-G^{-1}[0]}{1-\xi}\right\} \text{ (6)}
$$


Now, simplify these equations for the specific case $G[J] = \max(J, 0)$.


When $G[J] = \max(J, 0)$, using equations (5) and (6) above we know that $G[J]=J$ so it follows that $G[a^\mathrm{max}]$ and $G^{-1}[a^\mathrm{max}]=\frac{1}{a^\mathrm{max}}$. Assuming that $G^{-1}[0]=J_{th}$ then

$$
a=\frac{\frac{1}{a^\mathrm{max}}-J_{th}}{1-\xi} \text{ (7)}
$$

and

$$
J^\mathrm{bias}=J_{th}-\xi \left\{\frac{\frac{1}{a^\mathrm{max}}-J_{th}}{1-\xi} \right\} \text{ (8)}
$$

We also know that in the case of the ReLU rate approximation encoder, $J^{th}$ being the maximum current that results in a zero output rate corresponds directly with the $x-intercept$ $\xi$


**b) Neuron tuning curves.** Plot the neuron tuning curves $a_i(x)$ for 16 randomly generated neurons following the intercept and maximum rate distributions described above.


In [121]:
number_of_encoders = 16

Generate random $a^{max}$ uniformely distributed between 100Hz and 200Hz


In [122]:
low_freq = 100
high_freq = 200
num_a_max_samples = number_of_encoders
a_max_set = np.random.uniform(low_freq, high_freq, num_a_max_samples)

Generate random $x-intercepts$ $\xi$ uniformly distributed between -0.95 and 0.95

In [123]:
min_intercept = -0.95
max_intercept = 0.95
num_intercepts = number_of_encoders
intercept_set = np.random.uniform(min_intercept, max_intercept, num_intercepts)

Assign $J^{th}$ to intercept sets

In [124]:
J_th=intercept_set

Generate random signed encoders $e$

In [125]:
encoder_signs=(2*np.random.randint(0,2,size=(number_of_encoders))-1)

Generate a set of 16 random encoders $a_i(x)$ based on an basic encoder class 

In [126]:
def gain(a_max, x_intercept):
    inv_a_max = 1 / a_max
    numerator = inv_a_max - x_intercept
    demom = 1 - x_intercept
    return numerator / demom


def bias(a_max, x_intercept):
    gain_term = gain(a_max, x_intercept)
    return x_intercept - x_intercept * gain_term


class Encoder:
    def __init__(self, a_max, x_intercept, encoder_sign):
        self.a_max = a_max
        self.x_intercept = x_intercept
        self.encoder_sign = encoder_sign
        a = gain(a_max, x_intercept)
        j_bias = bias(a_max, x_intercept)
        self.a = a
        self.j_bias = j_bias


encoders = []

for i in range(number_of_encoders):
    encoders.append(Encoder(a_max_set[i], intercept_set[i], encoder_signs[i]))

**c) Computing identity decoders.** Compute the optimal identity decoder $\vec d$ for those 16 neurons (as shown in class). Report the value of the individual decoder coefficients. Compute $d$ using the matrix notation mentioned in the course notes. Do not apply any regularization. $A$ is the matrix of activities (the same data used to generate the plot in 1.1b).


In [127]:
# ✍ <YOUR SOLUTION HERE>

**d) Evaluating decoding errors.** Compute and plot $\hat{x}=\sum_i d_i a_i(x)$. Overlay on the plot the line $y=x$. Make a separate plot of $x-\hat{x}$ to see what the error looks like. Report the Root Mean Squared Error (RMSE) value.


In [128]:
# ✍ <YOUR SOLUTION HERE>

**e) Decoding under noise.** Now try decoding under noise. Add random normally distributed noise to $a$ and decode again. The noise is a random variable with mean $\mu=0$ and standard deviation of $\sigma=0.2 \max(A)$ (where $\max(A)$ is the maximum firing rate of all the neurons). Resample this variable for every different $x$ value for every different neuron. Create all the same plots as in part d). Report the RMSE.


In [129]:
# ✍ <YOUR SOLUTION HERE>

**f) Accounting for decoder noise.** Recompute the decoder $\vec d$ taking noise into account (i.e., apply the appropriate regularization, as shown in class). Show how these decoders behave when decoding both with and without noise added to $a$ by making the same plots as in d) and e). Report the RMSE for all cases.


In [130]:
# ✍ <YOUR SOLUTION HERE>

**g) Interpretation.** Show a 2x2 table of the four RMSE values reported in parts d), e), and f). This should show the effects of adding noise and whether the decoders $d$ are computed taking noise into account. Write a few sentences commenting on what the table shows, i.e., what the effect of adding noise to the activities is with respect to the measured error and why accounting for noise when computing the decoders increases/decreases/does not change the measured RMSE.


✍ \<YOUR SOLUTION HERE\>


## 1.2 Exploring sources of error

**a) Exploring error due to distortion and noise.** Plot the error due to distortion $E_\mathrm{dist}$ and the error due to noise $E_\mathrm{noise}$ as a function of $n$, the number of neurons. Generate two different loglog plots (one for each type of error) with $n$ values of at least $[4, 8, 16, 32, 64, 128, 256, 512]$. For each $n$ value, do at least $5$ runs and average the results. For each run, different $\alpha$, $J^\mathrm{bias}$, and $e$ values should be generated for each neuron. Compute $d$ taking noise into account, with $\sigma = 0.1 \max(A)$. Show visually that the errors are proportional to $1/n$ or $1/n^2$.


In [131]:
# ✍ <YOUR SOLUTION HERE>

**b) Adapting the noise level.** Repeat part a) with $\sigma = 0.01 \max(A)$.


In [132]:
# ✍ <YOUR SOLUTION HERE>

**c) Interpretation.** What does the difference between the graphs in a) and b) tell us about the sources of error in neural populations?


✍ \<YOUR SOLUTION HERE\>


## 1.3 Leaky Integrate-and-Fire neurons

**a) Computing gain and bias.** As in the second part of 1.1a), given a maximum firing rate $a^\mathrm{max}$ and a bias $J^\mathrm{bias}$, write down the equations for computing $\alpha$ and the $J^\mathrm{bias}$ for this specific neuron model.


✍ \<YOUR SOLUTION HERE\>


**b) Neuron tuning curves.** Generate the same plot as in 1.1b). Use $\tau_\mathrm{ref}=2 \mathrm{ms}$ and $\tau_{RC}=20 \mathrm{ms}$. Use the same distribution of $x$-intercepts and maximum firing rates as in 1.1.


In [133]:
# ✍ <YOUR SOLUTION HERE>

**c) Impact of noise.** Generate the same four plots as in 1.1f) (adding/not adding noise to $A$, accounting/not accounting for noise when computing $\vec d$), and report the RMSE both with and without noise.


In [134]:
# ✍ <YOUR SOLUTION HERE>

# 2. Reperesentation of Vectors

## 2.1 Vector tuning curves

**a) Plotting 2D tuning curves.** Plot the tuning curve of an LIF neuron whose 2D preferred direction vector is at an angle of $\theta=-\pi/4$, has an $x$-intercept at the origin $(0,0)$, and has a maximum firing rate of $100 \mathrm{Hz}$.


In [135]:
# ✍ <YOUR SOLUTION HERE>

**b) Plotting the 2D tuning curve along the unit circle.** Plot the tuning curve for the same neuron as in a), but only considering the points around the unit circle, i.e., sample the activation for different angles $\theta$. Fit a curve of the form $c_1 \cos(c_2\theta+c_3)+c_4$ to the tuning curve and plot it as well.


In [136]:
# ✍ <YOUR SOLUTION HERE>

**c) Discussion.** What makes a cosine a good choice for the curve fit in 2.1b? Why does it differ from the ideal curve?


✍ \<YOUR SOLUTION HERE\>


## 2.2 Vector representation

**a) Choosing encoding vectors.** Generate a set of $100$ random unit vectors uniformly distributed around the unit circle. These will be the encoders $\vec e$ for $100$ neurons. Plot these vectors with a quiver or line plot (i.e., not just points, but lines/arrows to the points).


In [137]:
# ✍ <YOUR SOLUTION HERE>

**b) Computing the identity decoder.** Use LIF neurons with the same properties as in question 1.3. When computing the decoders, take into account noise with $\sigma = 0.2\max(A)$. Plot the decoders in the same way you plotted the encoders.


In [138]:
# ✍ <YOUR SOLUTION HERE>

**c) Discussion.** How do these decoding vectors compare to the encoding vectors?


✍ \<YOUR SOLUTION HERE\>


**d) Testing the decoder.** Generate 20 random $\vec x$ values throughout the unit circle (i.e.,~with different directions and radiuses). For each $\vec x$ value, determine the neural activity $a_i$ for each of the 100 neurons. Now decode these values (i.e. compute $\hat{x} = D \vec a$) using the decoders from part b). Plot the original and decoded values on the same graph in different colours, and compute the RMSE.


In [139]:
# ✍ <YOUR SOLUTION HERE>

**e) Using encoders as decoders.** Repeat part d) but use the _encoders_ as decoders. This is what Georgopoulos used in his original approach to decoding information from populations of neurons. Plot the decoded values and compute the RMSE. In addition, recompute the RMSE in both cases, but ignore the magnitude of the decoded vectors by normalizing before computing the RMSE.


In [140]:
# ✍ <YOUR SOLUTION HERE>

**f) Discussion.** When computing the RMSE on the normalized vectors, using the encoders as decoders should result in a larger, yet still surprisingly small error. Thinking about random unit vectors in high dimensional spaces, why is this the case? What are the relative merits of these two approaches to decoding?


✍ \<YOUR SOLUTION HERE\>
